# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 9:59AM,246386,20,8516586,"ACI Healthcare USA, Inc.",Released
1,Sep 14 2022 9:59AM,246386,20,8521160,"ACI Healthcare USA, Inc.",Released
2,Sep 14 2022 9:59AM,246386,20,8521171,"ACI Healthcare USA, Inc.",Released
3,Sep 14 2022 9:59AM,246386,20,8522357,"ACI Healthcare USA, Inc.",Released
4,Sep 14 2022 9:59AM,246386,20,8522360,"ACI Healthcare USA, Inc.",Released
5,Sep 14 2022 9:59AM,246386,20,8522412,"ACI Healthcare USA, Inc.",Released
6,Sep 14 2022 9:59AM,246386,20,8524950,"ACI Healthcare USA, Inc.",Released
7,Sep 14 2022 9:58AM,246387,10,0085980158,ISDIN Corporation,Released
8,Sep 14 2022 9:58AM,246387,10,0085980160,ISDIN Corporation,Released
9,Sep 14 2022 9:58AM,246387,10,0085980159,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,246383,Released,4
40,246384,Released,37
41,246385,Released,2
42,246386,Released,7
43,246387,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246383,NaN,4.0
246384,NaN,37.0
246385,NaN,2.0
246386,NaN,7.0
246387,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246178,1.0,0.0
246182,0.0,1.0
246185,0.0,1.0
246195,1.0,0.0
246199,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246178,1,0
246182,0,1
246185,0,1
246195,1,0
246199,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246178,1,0
1,246182,0,1
2,246185,0,1
3,246195,1,0
4,246199,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246178,1,
1,246182,,1
2,246185,,1
3,246195,1,
4,246199,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc."
7,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation
13,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc."
50,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc"
52,Sep 14 2022 9:50AM,246383,10,Bio-PRF
56,Sep 14 2022 9:39AM,246382,12,Hush Hush
71,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.
78,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC"
79,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample
80,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc.",,7
1,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation,,6
2,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc.",,37
3,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc",,2
4,Sep 14 2022 9:50AM,246383,10,Bio-PRF,,4
5,Sep 14 2022 9:39AM,246382,12,Hush Hush,,15
6,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,,7
7,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",,1
8,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,,1
9,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc.",7,
1,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation,6,
2,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc.",37,
3,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc",2,
4,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4,
5,Sep 14 2022 9:39AM,246382,12,Hush Hush,15,
6,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,7,
7,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",1,
8,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,1,
9,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc.",7,
1,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation,6,
2,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc.",37,
3,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc",2,
4,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc.",7.0,NaN
1,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation,6.0,NaN
2,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc.",37.0,NaN
3,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc",2.0,NaN
4,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:59AM,246386,20,"ACI Healthcare USA, Inc.",7.0,0.0
1,Sep 14 2022 9:58AM,246387,10,ISDIN Corporation,6.0,0.0
2,Sep 14 2022 9:55AM,246384,15,"Alliance Pharma, Inc.",37.0,0.0
3,Sep 14 2022 9:52AM,246385,12,"Uniderm USA, Inc",2.0,0.0
4,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2956088,38.0,0.0
102,985422,4.0,0.0
12,492767,17.0,0.0
15,246384,37.0,0.0
16,246374,1.0,0.0
17,739102,3.0,0.0
19,739120,2.0,4.0
20,985351,10.0,0.0
21,3200771,10.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2956088,38.0,0.0
1,102,985422,4.0,0.0
2,12,492767,17.0,0.0
3,15,246384,37.0,0.0
4,16,246374,1.0,0.0
5,17,739102,3.0,0.0
6,19,739120,2.0,4.0
7,20,985351,10.0,0.0
8,21,3200771,10.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,38.0,0.0
1,102,4.0,0.0
2,12,17.0,0.0
3,15,37.0,0.0
4,16,1.0,0.0
5,17,3.0,0.0
6,19,2.0,4.0
7,20,10.0,0.0
8,21,10.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,102,Released,4.0
2,12,Released,17.0
3,15,Released,37.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21
Status,,,,,,,,,
Executing,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0
Released,38.0,4.0,17.0,37.0,1.0,3.0,2.0,10.0,10.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0
1,Released,38.0,4.0,17.0,37.0,1.0,3.0,2.0,10.0,10.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21
0,Executing,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0
1,Released,38.0,4.0,17.0,37.0,1.0,3.0,2.0,10.0,10.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()